In [1]:
#This scrapes all the years from the Billboard site
import time
from splinter import Browser
from bs4 import BeautifulSoup
from datetime import datetime
from contextlib import suppress
from selenium import webdriver
import json

from selenium.webdriver.chrome.options import Options     #These lines are to add uBlock origin to the headless browser
path_to_extension = '1.16.14_0'
chrome_options = Options()
chrome_options.add_argument('load-extension=' + path_to_extension) 
driver = webdriver.Chrome(chrome_options=chrome_options)
driver.create_options()
#driver.get("http://www.google.com")






# Initialize browser
def init_browser():
    executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
    return Browser("chrome", **executable_path, options=chrome_options, headless=False)
browser = init_browser()

urlList = []
urlListClean = []
year = ['1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', 
        '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', 
        '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', 
        '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', 
        '2016', '2017', '2018'] 



for y in year:
    # Visit billboard site
    url = "https://www.billboard.com/archive/charts/%s/hot-100" % y
    browser.visit(url)
    print(url)

    
    
    # Scrape page into soup
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    table = soup.find('table', class_ = 'archive-table')
    url = table.find_all('a', href = True)         #, class_ = 'sl-spot-list__ref'
    urlList.extend(url)
    for a in url:
        urlListClean.append(a['href'])
    
print(urlListClean)    
with open('URLs.json', 'w') as outfile:
      json.dump(urlListClean, outfile)   

In [1]:
#format the datetime from scrape
print(urlListClean[0])

from datetime import datetime

d = "1960-01-04"
date_obj = datetime.strptime(d, "%Y-%m-%d")
formatted_date = datetime.strftime(date_obj, "%B %d, %Y")
print(formatted_date)

NameError: name 'urlListClean' is not defined

In [2]:
#Second scrape to gather song data from the top ten artists for each week on the billboard site. 

# Visit billboard site

# with open("URLs.json","r") as f:
#     urlListClean = f.read()
#     print(urlListClean)[0]
    
# data = json.load('URLs.json')    

forJson = []

for u in urlListClean[2000:3040]:
    url = "https://www.billboard.com%s" %u
    browser.visit(url)
    print(url)

    
    
    # Scrape page into soup
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    #Find the top ten songs
    songs = []
    song_search = soup.find_all('span', class_="chart-list-item__title-text")
    for a in song_search[:9]:
        songs.append(a.text.strip())
        

    #Find the top ten artists
    artists = []
    artist_search = soup.find_all('div', class_="chart-list-item__artist")
    for a in artist_search[:9]:
        artists.append(a.text.strip())    
    
    #Weeks on chart
    weeks_on_chart = []
    chart_search = soup.find_all('div', class_="chart-list-item__weeks-on-chart")
    for a in chart_search[:9]:
        weeks_on_chart.append(a.text.strip())      
    
    weekOf = 0
    #print(weekOf)
    
    chart =[
                {'Week of': soup.find(class_ ="chart-detail-header__date-selector-button").text.strip() },
                {'Position': 1, 'Song':soup.find(class_ ="chart-number-one__title").text, 'Artist':(soup.find("div", class_="chart-number-one__artist").text.strip()), "Weeks on Chart":soup.find("div", class_="chart-number-one__weeks-on-chart").text},
                {'Position': 2, 'Song':songs[0], 'Artist':artists[0], 'Weeks on Chart': weeks_on_chart[0]},
                {'Position': 3, 'Song':songs[1], 'Artist':artists[1], 'Weeks on Chart': weeks_on_chart[1]},
                {'Position': 4, 'Song':songs[2], 'Artist':artists[2], 'Weeks on Chart': weeks_on_chart[2]},
                {'Position': 5, 'Song':songs[3], 'Artist':artists[3], 'Weeks on Chart': weeks_on_chart[3]},
                {'Position': 6, 'Song':songs[4], 'Artist':artists[4], 'Weeks on Chart': weeks_on_chart[4]},
                {'Position': 7, 'Song':songs[5], 'Artist':artists[5], 'Weeks on Chart': weeks_on_chart[5]},
                {'Position': 8, 'Song':songs[6], 'Artist':artists[6], 'Weeks on Chart': weeks_on_chart[6]},
                {'Position': 9, 'Song':songs[7], 'Artist':artists[7], 'Weeks on Chart': weeks_on_chart[7]},
                {'Position': 10, 'Song':songs[8], 'Artist':artists[8], 'Weeks on Chart': weeks_on_chart[8]}]

    week_total = {weekOf : chart}
    #print(week_total)
    forJson.append(week_total)
    
#Just put it to a json when it's done
with open('billboard[2000:3040].json', 'a') as outfile:      #'a' so it appends the file
   json.dump(forJson, outfile)
    
print('2000:3040')    

In [3]:
#Just put it to a json when it's done
with open('billboard.json', 'a') as outfile:      #'a' so it appends the file
   json.dump(forJson, outfile)
    
print('k')   